In [ ]:
import json
import os

#### parameters that needs to be exhanged between the sender and the receiver #####
IP = "0.0.0.0"
PORT = 23422
#################################################################################### 

while True:
   os.system('python3 rx.py ' + "\'" + json.dumps({'IP': IP, 'PORT': PORT}) + "\'")
